In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://four-points.marriott.com/hotel-directory/"
driver.get(url)
# viewhotels_button = driver.find_elements_by_xpath('/html/body/div[1]/div[1]/div[5]/div[4]/a')[0]
# viewhotels_button.click()
# northamhotels_button = driver.find_elements_by_xpath('/html/body/div[1]/div[1]/div[5]/div[3]/dl/dt[7]/a')[0]
# northamhotels_button.click()

In [3]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
stuff = soup.find_all('a', {'class', 'property'})
# stuff
hotel_list = []
for x in stuff:
    hotel = {}
    hotel['name'] = x.text.strip()
    hotel['url'] = x.attrs['href']
    hotel_list.append(hotel)
hotelsurl_df = pd.DataFrame(hotel_list)
hotelsurl_df

,name,url
0,Four Points by Sheraton Oran,https://www.marriott.com/ORNFP
1,Four Points by Sheraton Nairobi Airport,https://www.marriott.com/NBOFA
2,Four Points by Sheraton Nairobi Hurlingham,https://www.marriott.com/NBOFP
3,Four Points by Sheraton Lagos,https://www.marriott.com/LOSFP
4,"Four Points by Sheraton Arusha, The Arusha Hotel",https://www.marriott.com/JROFP
...,...,...
303,Four Points by Sheraton Bogota,https://www.marriott.com/BOGFP
304,Four Points by Sheraton Cali,https://www.marriott.com/CLOFP
305,Four Points by Sheraton Medellin,https://www.marriott.com/MDEFP
306,Four Points by Sheraton Cuenca,https://www.marriott.com/CUEFP


In [5]:
%%time

hotel_list = []
counter = 0
for index, row in hotelsurl_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'FourPoints'
        except:
            hotel['brand'] = 'FourPoints'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df   

{'brand': 'FourPoints', 'name': 'Four Points by Sheraton Oran', 'url': 'https://www.marriott.com/ORNFP', 'street': 'Boulevard du 19 Mars, Route des falaises', 'locality': 'Oran', 'state': '', 'postalcode': '31000', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/ornfp-four-points-oran/?directPageRequest=true', 'coordinate3': '35.716661', 'coordinate4': '-0.610162', 'description': "[ Make your escape to the serene Mediterranean coast at Four Points by Sheraton Oran. Located in the historic Algerian port city of Oran, our luxury hotel boasts modern rooms, world-class dining, a rooftop pool and premium hotel amenities and service. Conveniently located near local beaches, our hotel also offers easy access to popular Oran sights such as Fort Santa Cruz. Choose from well-appointed rooms with luxury bedding, modern entertainment systems, high-speed Wi-Fi and floor-to-ceiling windows overlooking Oran and the sea. Enjoy a dazzling array of restaurant and ba

{'brand': 'FourPoints', 'name': 'Four Points by Sheraton Dar es Salaam New Africa', 'url': 'https://www.marriott.com/DARFP', 'street': 'Azikiwe Street/Sokoine Drive Mchafukoge, PO Box 9314', 'locality': 'Dar es Salaam', 'state': '', 'postalcode': '9314', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/darfp-four-points-dar-es-salaam-new-africa/?directPageRequest=true', 'coordinate2': '-6.817006', 'coordinate3': '39.287936', 'coordinate4': 'N/A', 'description': '[ Right in the heart of Tanzania’s capital city, with views of Dar es Salaam Harbour and close to the Central Business District. Four Points by Sheraton Dar es Salaam New Africa Hotel is the premier Conference and Meetings Venue in Tanzania. With 12 Meeting Venues and capacity for up to 300 persons in one venue space, your every business need can be met in comfort. Situated nearby the Zanzibar ferry, our hotel is the perfect starting point for everyone who wants to explore the city and beyond. Soak in the city’s vibr

{'brand': 'FourPoints', 'name': 'Four Points by Sheraton Guangzhou, Dongpu', 'url': 'https://www.marriott.com/CANFP', 'street': 'No. 1 Jing Ying Road, Hui Cai Road, Dongpu, Tian He District', 'locality': 'Guangzhou', 'state': '', 'postalcode': '510660', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/canfp-four-points-guangzhou-dongpu/?directPageRequest=true', 'coordinate3': '23.113478', 'coordinate4': '113.417068', 'description': "[ We have 296 spacious guestrooms, Four Points by Sheraton Guangzhou, Dongpu has what matters most to you; like a comfortable bed, a delicious breakfast and fresh coffee, as well as great local beer with BestBrewsTM. Each of our spacious rooms comes equipped with free WiFi, a flat-screen TV and more and room service is available 24/7. When you're hungry, you need not venture far, our hotel features 2 sleep restaurants to sate your appetite. The property also offers a heated indoor pool, an elegant spa and a fitness cente

{'brand': 'FourPoints', 'name': 'Four Points by Sheraton Guilin, Lingui', 'url': 'https://www.marriott.com/KWLFP', 'street': 'No.1 Xicheng South Road, Lingui District', 'locality': 'Guilin', 'state': '', 'postalcode': '541100', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/kwlfp-four-points-guilin-lingui/?directPageRequest=true', 'coordinate3': '25.200004', 'coordinate4': '110.199997', 'description': '[ Business, Leisure or Both. The Four Points by Sheraton Guilin, Lingui offers uncomplicated comforts and a convenient location in the heart of Lingui, Guilin’s new business and administrative district. Our hotel’s complex includes a retail mall with government and corporate offices nearby. Plenty of Options. Take a dip in the indoor heated pool or work up a sweat in the health club. Enjoy wireless High-Speed Internet Access in our lobby. That’s also where you’ll find our lounge, where you can relax with a freshly-brewed coffee or sample the Interna

{'brand': 'FourPoints', 'name': 'Four Points by Sheraton Shenzhou Peninsula', 'url': 'https://www.marriott.com/SYXPF', 'street': 'Shenzhou Peninsula Resort District', 'locality': 'Wanning', 'state': '', 'postalcode': '571528', 'coordinate1': '110.355302853', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Located on the beautiful cove - Festival Beach of Hainan’s Shenzhou Peninsula, our resort features a comfortable subtropical climate year-round. The hotel sits by the mountains and borders on the coastline, only steps away from the PGA Level golf course – The Dunes, designed by internationally-renowned Tom Weiskopf. Nearby, Shenzhou Beach Club, a one-stop premium water sport. They provide our guests with a comfortable resort with never-ending recreation and sports activities. Plus the fully equipped Kids Club ensures that children also have their fair share of good fun at the beach resorts. The Four Points by Sheraton resort in Greater China, Four P

...

{'brand': 'FourPoints', 'name': 'Four Points by Sheraton Salt Lake City Airport', 'url': 'https://www.marriott.com/SLCFP', 'street': '200 N Admiral Byrd Road', 'locality': 'Salt Lake City', 'state': 'Utah', 'postalcode': '84116', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/slcfp-four-points-salt-lake-city-airport/?directPageRequest=true', 'coordinate3': '40.772381', 'coordinate4': '-112.021591', 'description': "[ The Four Points Marriott is in full compliance with rigorous brand cleaning, operational standards and CDC guidelines of social distancing, face coverings and hand sanitizing. We also endorsed the AHLA Stay Safe program and pledged with Utah's Stay Safe program. Blending timeless comfort with modern amenities, the Four Points offers an unrivaled stay for business and leisure travelers alike. Depart Salt Lake City International Airport in our complimentary shuttle and embark on an effortless getaway. Located along I-80, our hotel offers

{'brand': 'FourPoints', 'name': 'Four Points by Sheraton Seattle Airport South', 'url': 'https://www.marriott.com/SEASF', 'street': '22406 Pacific Highway South', 'locality': 'Des Moines', 'state': 'Washington', 'postalcode': '98198', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/seasf-four-points-seattle-airport-south/?directPageRequest=true', 'coordinate3': '47.400629', 'coordinate4': '-122.296834', 'description': "[ Maintain your balance at Four Points by Sheraton Seattle Airport South. Conveniently located near Sea-Tac Airport and South Puget Sound, our hotel offers both productive and relaxing spaces for business and leisure. Recharge in our spacious hotel rooms, featuring ultra-comfortable beds, an in-room mini fridge, and a laptop-sized safe. Stay satisfied with authentic American cuisine at Olympus Grill, our onsite restaurant serving breakfast and dinner. Local South Seattle restaurants are also a convenient option, with choices like Wal

{'brand': 'FourPoints', 'name': 'Four Points by Sheraton Curitiba', 'url': 'https://www.marriott.com/CWBFP', 'street': 'Av. Sete de Setembro 4211, Agua Verde', 'locality': 'Curitiba', 'state': '', 'postalcode': '80250-210', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/cwbfp-four-points-curitiba/?directPageRequest=true', 'coordinate2': '-25.43', 'coordinate3': '-49.285', 'coordinate4': 'N/A', 'description': '[ The Four Points by Sheraton Curitiba offers high standard of sophistication and modernity in the hotel business. Located in the prime downtown area, near by the best shopping malls, restaurants and business centers in the city. The hotel is equipped with fire protection system, consisting of smoke detectors and sprinklers. For moments of leisure and well-being, the hotel offers semi-Olympic swimming pool (covered and heated), solarium, sauna and steam rooms, fitness center and the Tereza Zanchi Spa, with various therapies and relaxation options. The hotel offers six

{'brand': 'FourPoints', 'name': 'Four Points by Sheraton Cuenca', 'url': 'https://www.marriott.com/CUEFP', 'street': 'Ave. Circunvalación Sur y Ave, Felipe II Junto al Mall del Río', 'locality': 'Cuenca', 'state': '', 'postalcode': '010169', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/cuefp-four-points-cuenca/?directPageRequest=true', 'coordinate2': '-2.920799', 'coordinate3': '-79.016645', 'coordinate4': 'N/A', 'description': "[ Just a few kilometers from the renowned heritage of Cuenca's city center, our hotel is steps from the biggest shopping mall and Convention Center. ]", 'overall_rating': 'N/A', 'cleanliness_rating': '4.9', 'service_rating': '4.8'}

2020-09-21 22:52:41.340852

Collected 307 hotels.
{'brand': 'FourPoints', 'name': 'Four Points by Sheraton Miraflores', 'url': 'https://www.marriott.com/LIMFP', 'street': 'Calle Alcanfores 290 Miraflores', 'locality': 'Lima', 'state': '', 'postalcode': '', 'coordinate1': '-12.128621', 'coordinate2': '-77.028172', 'coo

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,FourPoints,Four Points by Sheraton Oran,https://www.marriott.com/ORNFP,"Boulevard du 19 Mars, Route des falaises",Oran,,31000,Reviews,https://www.marriott.com/hotels/maps/travel/or...,35.716661,-0.610162,[ Make your escape to the serene Mediterranean...,N/A,4.2,3.9
1,FourPoints,Four Points by Sheraton Nairobi Airport,https://www.marriott.com/NBOFA,"9042/1044 Tower Avenue, Jomo Kenyatta Intl Air...",Nairobi,,1158-00606,-1.339775,36.918893,N/A,N/A,[ Whether you’re on business and needing a fun...,N/A,4.8,4.7
2,FourPoints,Four Points by Sheraton Nairobi Hurlingham,https://www.marriott.com/NBOFP,Argwings Kodhek Road,Nairobi,,,Reviews,https://www.marriott.com/hotels/maps/travel/nb...,-1.2925,36.7887,[ We offer exceptional Kenyan hospitality and ...,N/A,4.5,4.5
3,FourPoints,Four Points by Sheraton Lagos,https://www.marriott.com/LOSFP,"Plot 9/10 Block 2 Oniru Chieftaincy Estate, Vi...",Lagos,,,https://www.marriott.com/hotels/maps/travel/lo...,6.433611,3.443125,N/A,[ Enjoy effortless style with the essentials d...,N/A,4.0,3.8
4,FourPoints,"Four Points by Sheraton Arusha, The Arusha Hotel",https://www.marriott.com/JROFP,P.O. Box 88,Arusha,,,Reviews,https://www.marriott.com/hotels/maps/travel/jr...,-3.372792,36.69445,"[ Established in 1894, the Four Points by Sher...",N/A,4.8,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,FourPoints,Four Points by Sheraton Bogota,https://www.marriott.com/BOGFP,Carrera 18 #93A - 83,Bogota,,20001,Reviews,https://www.marriott.com/hotels/maps/travel/bo...,4.679802,-74.052948,"[ Four Points By Sheraton Bogota, is a © Hotel...",N/A,4.5,4.4
304,FourPoints,Four Points by Sheraton Cali,https://www.marriott.com/CLOFP,Cali 18 North No. 4 N-08,Cali,,,Reviews,https://www.marriott.com/hotels/maps/travel/cl...,3.459417,-76.5298,[ Four Points by Sheraton Cali is a modern hot...,N/A,4.7,4.7
305,FourPoints,Four Points by Sheraton Medellin,https://www.marriott.com/MDEFP,Carrera 43 C #6 Sur 100,Medellin,,,https://www.marriott.com/hotels/maps/travel/md...,6.199227,-75.576228,N/A,[ Four Points by Sheraton Medellin offers its ...,N/A,4.7,4.5
306,FourPoints,Four Points by Sheraton Cuenca,https://www.marriott.com/CUEFP,"Ave. Circunvalación Sur y Ave, Felipe II Junto...",Cuenca,,010169,https://www.marriott.com/hotels/maps/travel/cu...,-2.920799,-79.016645,N/A,[ Just a few kilometers from the renowned heri...,N/A,4.9,4.8


In [6]:
hotel_df = hotel_df.drop_duplicates(subset='name')
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,FourPoints,Four Points by Sheraton Oran,https://www.marriott.com/ORNFP,"Boulevard du 19 Mars, Route des falaises",Oran,,31000,Reviews,https://www.marriott.com/hotels/maps/travel/or...,35.716661,-0.610162,[ Make your escape to the serene Mediterranean...,N/A,4.2,3.9
1,FourPoints,Four Points by Sheraton Nairobi Airport,https://www.marriott.com/NBOFA,"9042/1044 Tower Avenue, Jomo Kenyatta Intl Air...",Nairobi,,1158-00606,-1.339775,36.918893,N/A,N/A,[ Whether you’re on business and needing a fun...,N/A,4.8,4.7
2,FourPoints,Four Points by Sheraton Nairobi Hurlingham,https://www.marriott.com/NBOFP,Argwings Kodhek Road,Nairobi,,,Reviews,https://www.marriott.com/hotels/maps/travel/nb...,-1.2925,36.7887,[ We offer exceptional Kenyan hospitality and ...,N/A,4.5,4.5
3,FourPoints,Four Points by Sheraton Lagos,https://www.marriott.com/LOSFP,"Plot 9/10 Block 2 Oniru Chieftaincy Estate, Vi...",Lagos,,,https://www.marriott.com/hotels/maps/travel/lo...,6.433611,3.443125,N/A,[ Enjoy effortless style with the essentials d...,N/A,4.0,3.8
4,FourPoints,"Four Points by Sheraton Arusha, The Arusha Hotel",https://www.marriott.com/JROFP,P.O. Box 88,Arusha,,,Reviews,https://www.marriott.com/hotels/maps/travel/jr...,-3.372792,36.69445,"[ Established in 1894, the Four Points by Sher...",N/A,4.8,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,FourPoints,Four Points by Sheraton Bogota,https://www.marriott.com/BOGFP,Carrera 18 #93A - 83,Bogota,,20001,Reviews,https://www.marriott.com/hotels/maps/travel/bo...,4.679802,-74.052948,"[ Four Points By Sheraton Bogota, is a © Hotel...",N/A,4.5,4.4
304,FourPoints,Four Points by Sheraton Cali,https://www.marriott.com/CLOFP,Cali 18 North No. 4 N-08,Cali,,,Reviews,https://www.marriott.com/hotels/maps/travel/cl...,3.459417,-76.5298,[ Four Points by Sheraton Cali is a modern hot...,N/A,4.7,4.7
305,FourPoints,Four Points by Sheraton Medellin,https://www.marriott.com/MDEFP,Carrera 43 C #6 Sur 100,Medellin,,,https://www.marriott.com/hotels/maps/travel/md...,6.199227,-75.576228,N/A,[ Four Points by Sheraton Medellin offers its ...,N/A,4.7,4.5
306,FourPoints,Four Points by Sheraton Cuenca,https://www.marriott.com/CUEFP,"Ave. Circunvalación Sur y Ave, Felipe II Junto...",Cuenca,,010169,https://www.marriott.com/hotels/maps/travel/cu...,-2.920799,-79.016645,N/A,[ Just a few kilometers from the renowned heri...,N/A,4.9,4.8


In [7]:
hotel_df.to_csv('fourpoints_description.csv')

In [8]:
# driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
# url = 'https://www.marriott.com/hotels/travel/bhmpw-four-points-birmingham-homewood/'
# driver.get(url)
# page_source = driver.page_source
# soup = BeautifulSoup(page_source, 'lxml')

In [9]:
# stuff = soup.find('div', {'class' : 'l-container l-header-container'})
# stuff

In [10]:
# hotel_list = []
# hotel = {}
# hotel['name'] = stuff.find_all('span')[0].text
# hotel['street'] = stuff.find_all('span')[2].text
# hotel['locality'] = stuff.find_all('span')[4].text
# hotel['state'] = stuff.find_all('span')[6].text
# hotel['postalcode'] = stuff.find_all('span')[7].text
# hotel['latitude'] = stuff.find_all('span')[26].text
# hotel['longitude'] = stuff.find_all('span')[27].text
# hotel_list.append(hotel)
# hotel_list

In [11]:
# desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})

In [12]:
# str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')

In [13]:
# for x in stuff.find_all('div'):
# #     name = x.find('a').text
#     print(name)

In [14]:
# %%time

# hotel_buttons = driver.find_elements_by_class_name("property")
# original_window = driver.current_window_handle
# wait = WebDriverWait(driver, 5)
# hotel_list =[]
# for x in range(len(hotel_buttons)):

#     try:
#         if hotel_buttons[x].is_displayed():
#             driver.execute_script("arguments[0].click();", hotel_buttons[x])
#             wait.until(EC.number_of_windows_to_be(2))
#             for window_handle in driver.window_handles:
#                 if window_handle != original_window:
#                     driver.switch_to.window(window_handle)
#     #         driver.switch_to.window(tab.get(x))
#     #         currentURL = driver.current_url
#     #         print(driver.window_handles)
#             time.sleep(5)
#             page_source = driver.page_source
#             soup = BeautifulSoup(page_source, 'lxml')
#             stuff = soup.find('div', {'class' : 'l-container l-header-container'})
# #             hotel_list = []
#             hotel = {}
#             try:
#                 hotel['name'] = stuff.find_all('span')[0].text
#             except:
#                 hotel['name'] = 'N/A'
#             try:
#                 hotel['street'] = stuff.find_all('span')[2].text
#             except:
#                 hotel['street'] = 'N/A'
#             try:
#                 hotel['locality'] = stuff.find_all('span')[4].text
#             except:
#                 hotel['locality'] = 'N/A'        
#             try:
#                 hotel['state'] = stuff.find_all('span')[6].text
#             except:
#                 hotel['state'] = 'N/A'            
#             try:
#                 hotel['postalcode'] = stuff.find_all('span')[7].text
#             except:
#                 hotel['postalcode'] = 'N/A'            
#             try:
#                 hotel['latitude'] = stuff.find_all('span')[26].text
#             except:
#                 hotel['latitude'] = 'N/A'            
#             try:
#                 hotel['longitude'] = stuff.find_all('span')[27].text
#             except:
#                 hotel['longitude'] = 'N/A'            
#             desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
#             try:
#                 hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
#             except:
#                 hotel['description'] = 'N/A'
#             hotel_list.append(hotel)
#             print(hotel)
#             print(datetime.now())
#             print('='*80)
#             print('='*80)
#             print()
#             driver.close()
#             driver.switch_to.window(original_window)
#             time.sleep(1)

#             print(len(hotel_list))
#     except:
#         print(f"Number {x} didn't work at {datetime.now()}")
#         continue

# fourpoint_df = pd.DataFrame(hotel_list)
# fourpoint_df